In [4]:
from openai import OpenAI
from tqdm import tqdm
import os

os.environ['HTTP_PROXY'] = 'http://127.0.0.1:5035'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:5035'

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="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"
)


In [10]:
# pdf到文本
import pdfplumber
from tqdm import tqdm

def extract_text_from_pdf(pdf_path, txt_path):
    with pdfplumber.open(pdf_path) as pdf, open(txt_path, 'w', encoding='utf-8') as file:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                file.write(text + '\n')

# 使用例子
extract_text_from_pdf('./23会计专硕文献综述/23会硕01时国军.pdf',
                       'output.txt')

def process_all_pdfs(input_dir, output_dir):
    for filename in tqdm(os.listdir(input_dir)):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(input_dir, filename)
            txt_filename = (os.path.splitext(filename)[0] + '.txt').replace(' ','').replace('+','')
            txt_path = os.path.join(output_dir, txt_filename)
            extract_text_from_pdf(pdf_path, txt_path)


process_all_pdfs('23会计专硕文献综述','23会计专硕文献综述_md')

100%|██████████| 51/51 [00:48<00:00,  1.06it/s]


In [22]:
def trim_after_reference(text):
    reference_keyword = "参考文献"
    a_length = len(text) // 3
    start = 0

    while True:
        # 查找“参考文献”的位置
        pos = text.find(reference_keyword, start)
        
        # 如果没有找到或者位置在文件的一半之前，更新搜索的起始位置
        if pos == -1 or pos < a_length:
            if pos == -1:
                break
            start = pos + 1
        else:
            # 找到了位于文件一半之后的“参考文献”，去掉这个位置之后的内容
            return text[:pos]
    
    # 如果没有找到合适的“参考文献”，返回原字符串
    return text


def process_txt_files(input_dir, output_dir):
    for filename in os.listdir(input_dir):
        if filename.lower().endswith('.txt'):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)

            with open(input_path, 'r', encoding='utf-8') as file:
                content = file.read()

            trimmed_content = trim_after_reference(content)

            with open(output_path, 'w', encoding='utf-8') as file:
                file.write(trimmed_content)

In [21]:
process_txt_files('23会计专硕文献综述_md','23会计专硕文献综述_trimed')

In [7]:
def grade(text,temperature=0.2, show = True):

    system_prompt = f"""你是一位经济学教师，用户会给你提供一份经济学的文献综述作业，请你进行评价。

    -要求从以下5个方面进行评价。
1. 文献的广度和深度：评估所涉及的文献是否全面覆盖了研究主题相关的各个方面，并且是否展示了对这些文献的深入理解。
2. 批判性分析：检查文献综述是否进行了文献的批判性分析，包括对研究方法、结论的有效性及其对领域贡献的评价。
3. 逻辑结构和组织：评估文献综述的组织结构是否清晰，信息是否按照逻辑顺序排列，以及是否有助于读者理解研究主题。
4. 写作质量：检查文献综述的写作是否清晰、准确、无语法或拼写错误，以及是否使用了适当的学术语言和风格。
5. 综合性和原创性：评估文献综述在整合不同研究和观点方面的能力，以及作者是否能在总结现有文献的基础上提出新的见解或理论框架。

- 输出要求，写成一段话：
首先陈述作业的主题，即这是一篇针对什么问题的综述。然后从上述5个方面进行评价。最后给出总评，满分100分。如“综合评价: 75分”

- 范例：

这份文献综述探讨了经济全球化对发展中国家经济增长的影响。在文献的广度和深度方面，作者广泛引用了关键的经济理论和案例研究，如通过对比Krugman和Stiglitz的观点来阐述不同经济学派的看法，但在涉及最新研究时略显不足，缺乏对近五年内相关领域研究的引用。在批判性分析方面，作者对引用文献的方法论进行了评估，如对Smith的案例研究进行了深入分析，指出了其样本选择的局限性，但在讨论不同观点时，未能充分展现各观点间的联系和差异。在写作质量方面，文献综述的语言清晰，专业术语使用得当，但偶尔出现较长的句子，影响了阅读流畅性。综合性和原创性方面，文献综述成功整合了不同的理论和实证研究，提出了发展中国家在经济全球化过程中可能面临的新挑战，但在提出创新性见解方面略显不足。

综合评价: 75分
    """.strip()

    stream = client.chat.completions.create(
        extra_headers={
            "HTTP-Referer": "py4ss.net", # Optional, for including your app on openrouter.ai rankings.
            "X-Title": 'transltion', # Optional. Shows in rankings on openrouter.ai.
        },
        model="openai/gpt-3.5-turbo",
        messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": text}],
        stream=True,
        temperature=temperature,
        frequency_penalty=0.8
    )

    result = ''

    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            if show: print(chunk.choices[0].delta.content, end="")
            result += chunk.choices[0].delta.content

    return result


with open('./23会计专硕文献综述_trimed/23会硕01时国军.txt') as f:
    text = f.read()

result = grade(text)


这份文献综述探讨了内部控制对财务风险的影响。在文献的广度和深度方面，作者对国内外相关文献进行了梳理和分析，涵盖了内部控制在防范和降低财务风险方面的重要作用。在批判性分析方面，作者评估了引用文献的有效性，并指出了一些研究中存在的限制和不足之处。在逻辑结构和组织方面，文献综述具有清晰的组织结构，并按照逻辑顺序排列信息，有助于读者理解研究主题。在写作质量方面，文献综述使用了准确无误、清晰流畅的语言，并避免了语法或拼写错误。在综合性和原创性方面，文献综述成功整合了不同观点和实证研究，并提出未来需要进一步探讨的问题。然而，在提出新见解或理论框架方面还有待改进。

综合评价: 85分

In [16]:
import pandas as pd

result = []

input_dir = './23会计专硕文献综述_trimed'
for filename in tqdm(os.listdir(input_dir)):
    if filename.lower().endswith('.txt'):
        input_path = os.path.join(input_dir, filename)
        with open(input_path, 'r', encoding='utf-8') as file:
                        content = file.read()
        grade_result = grade(content,show=False)

        result.append({'title':filename.replace('.txt',''),
                       'result':grade_result,
                       'grade':grade_result[-3:]})
        

pd.DataFrame(result).sort_values('title').to_excel('文献综述打分.xlsx',index=False)


100%|██████████| 51/51 [11:03<00:00, 13.00s/it]


In [17]:
pd.read_excel('文献综述打分.xlsx')

,title,result,grade
0,23会硕01时国军,这份文献综述探讨了内部控制对财务风险的影响。在文献的广度和深度方面，作者对国内外相关文献进行...,85分
1,23会硕02上官靖悦,这份文献综述探讨了企业高派现股利政策的动因和效果。在文献的广度和深度方面，作者涵盖了国内外学...,85分
2,23会硕03曾姝婷,文献的广度和深度：该文献综述对大数据审计领域的研究现状和前沿动态进行了梳理和归纳，涵盖了数据...,80分
3,23会硕04舒元,这份文献综述探讨了可视化技术对审计线索挖掘的影响。在文献的广度和深度方面，作者回顾了当前审计...,80分
4,23会硕05杨颂可,这份文献综述探讨了去杠杆与房地产企业财务风险的关系。在文献的广度和深度方面，作者引用了多个相...,85分
5,23会硕06吴家欣,这份文献综述探讨了企业并购绩效的影响因素。在文献的广度和深度方面，作者对国内外相关文献进行了...,80分
6,23会硕07卜湘玉,这份文献综述探讨了公司治理结构对内部控制的影响。在文献的广度和深度方面，作者引用了国内外相关...,80分
7,23会硕08陈晓彤,综合评价: 80分\n\n1. 文献的广度和深度：文献综述涵盖了数字经济对养老产业发展的影响...,空间。
8,23会硕09杨小凤,这份文献综述探讨了数字化转型下企业司库建设的现状分析。在文献的广度和深度方面，作者回顾了近年...,65分
9,23会硕10胡梦宇,这份文献综述探讨了管理层收购对企业的影响。在文献的广度和深度方面，作者引用了国内外众多学者的...,80分
